# モデルの保存方法
- 詳しくは公式読め
- 一応参考に: https://www.oio-blog.com/contents/pytorch-advanced

まずモデルの保存を行う目的は２つあります。

- 学習済みモデルを使って推論を行う
- 保存済みモデルの学習を再開する

目的によって、保存しておくべき内容が違います。
次に PyTorch でモデルを保存する方法について確認してきます。
PyTorch ではモデルを保存する方法が 2 通りあります。

- モデル全体を保存する
- モデルのパラメータを保存する

さらに保存する際には GPU か CPU なのかを注意する必要があります。
ややこしいですが、認識しておく必要があります。

In [ ]:
# 以下はサンプル例かつ非推奨。ここで実行してもエラーになる
# net.apply(init_weights)　# 追加
# net.train() # 学習

# 追加部分
dir_name = 'output'

if not os.path.exists(dir_name):
    os.mkdir(dir_name)

model_save_path = os.path.join(dir_name, "model_full.pt")

# モデル保存
torch.save(net, model_save_path)

# モデルロード
model_full = torch.load(model_save_path)

- 上の例は、gpuで保存時、次にロードする時gpuでないとまずいらしい
- しかも保存時のサイズがでかい
- なので、以下推奨

In [ ]:
# モデル保存
# torch.save(net.state_dict(), model_save_path) # これもgpuで保存している（一応読み出す方法はある）
torch.save(net.to('cpu').state_dict(), model_save_path) # なので一度cpu にする

# モデルロード
model_cpu.load_state_dict(torch.load(model_save_path))

checkpointを作り、再学習のためモデルを保存する場合

In [ ]:
# 保存側
if epoch % 3 == 0: # 好きなタイミングで止める
        file_name = 'epoch_{}.pt'.format(epoch)
        path = os.path.join(checkPoint_dir, file_name)
        torch.save({
            'epoch' : epoch,
            'model_state_dict' : net.state_dict(),
            'optimaizer_state_dict': optimizer.state_dict(),
            'loss': avg_train_loss
        }, path)

In [ ]:
# 再学習側
tmp_path = 'checkPoint/epoch_3.pt'

if os.path.exists(tmp_path):
    checkpoint = torch.load(tmp_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimaizer_state_dict'])
    epoch_num = checkpoint['epoch']
    loss = checkpoint['loss']

## おまけ1: register_buffer
- 論文実装の際にたまに使われる
- model のパラメーター ではないけどモデルに持っておきたい値を保存する際に使う
- 例:  Batchnormalization の計算のためのバッチごとの計算結果を保持するのに使われます。
    - 疑問: もしかして普通のBNのgamma, betaは保存されないの？
- 使い方は知らん